#### 1. Library Install & Load / Basic Setting

In [1]:
!git clone https://github.com/UKPLab/sentence-transformers.git

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 11495, done.
remote: Counting objects: 100% (3598/3598), done.
remote: Compressing objects: 100% (943/943), done.
remote: Total 11495 (delta 3137), reused 2663 (delta 2655), pack-reused 7897 (from 1)
Receiving objects: 100% (11495/11495), 25.93 MiB | 29.67 MiB/s, done.
Resolving deltas: 100% (8174/8174), done.


In [2]:
!git clone https://github.com/kakaobrain/kor-nlu-datasets.git

Cloning into 'kor-nlu-datasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 16 (delta 1), reused 1 (delta 1), pack-reused 14 (from 1)
Receiving objects: 100% (16/16), 46.67 MiB | 24.03 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
!pip uninstall sentence_transformers

In [6]:
%cd sentence-transformers

/content/sentence-transformers


In [7]:
!pip install -e .

Obtaining file:///content/sentence-transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sentence-transformers (pyproject.toml) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-3.4.0.dev0-0.editable-py3-none-any.whl size=10980 sha256=73220f1567f2afe68315fa28ec57babd442c75a1567bf92d29975f1911702433
  Stored in directory: /tmp/pip-ephem-wheel-cache-48md_b2_/wheels/20/79/ec/b839e68c83b7f73dfbf15cf4f989275c8d4eed6b3e247eecbd
Successfully built sentence-transformers


In [8]:
import sys
import os

# 현재 워킹 디렉토리를 /content로 변경
os.chdir("/content")

# sentence-transformers 폴더의 경로를 sys.path에 추가
sys.path.append("/content/sentence-transformers")

In [9]:
import os
import csv
import math
import torch
import random
import logging
import numpy as np

from datasets import Dataset
from datetime import datetime
from torch.utils.data import DataLoader

from sentence_transformers import (
    SentenceTransformer,
    LoggingHandler,
    datasets,
    models,
    losses,
)
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction
from sentence_transformers.readers import InputExample

In [10]:
# 로거 설정
log_file = "log_output.txt"

logging.basicConfig(
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[
        logging.FileHandler(log_file),  # 파일 핸들러로 로그를 파일에 기록
        logging.StreamHandler()        # 콘솔에도 출력
    ]
)

# 시드 고정
SEED = 777
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [11]:
# 데이터 경로 설정
KORNLI_PATH = '/content/kor-nlu-datasets/KorNLI'
KORSTS_PATH = '/content/kor-nlu-datasets/KorSTS'
OUTPUT_DIR = 'output'

In [12]:
final_scores = {}

In [13]:
logging.info("설정 및 라이브러리, 데이터 git 복제 완료.")

In [14]:
# KorSTS 데이터셋을 읽어와 InputExample 리스트를 반환
def load_kor_sts_samples(filename):
    samples = []
    with open(filename, 'rt', encoding='utf-8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            score = float(row['score']) / 5.0 # Normalize
            samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
    return samples

In [15]:
def load_kor_nli_samples(filename):
    """
    KorNLI 데이터를 읽어와서, Entailment vs Contradiction를 이용해
    MultipleNegativesRankingLoss에 맞는 형태(InputExample)를 생성.
    """
    data = {}
    def add_to_samples(sent1, sent2, label):
        if sent1 not in data:
            data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
        data[sent1][label].add(sent2)

    with open(filename, 'r', encoding='utf-8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            sent1 = row['sentence1'].strip()
            sent2 = row['sentence2'].strip()
            gold_label = row['gold_label'].strip()
            if gold_label not in ['entailment','contradiction','neutral']:
                # 혹시 라벨이 이상하면 건너뛰기
                continue
            add_to_samples(sent1, sent2, gold_label)
            add_to_samples(sent2, sent1, gold_label)

    samples = []
    for sent1, labels_dict in data.items():
        # entailment/contradiction이 각각 하나 이상 있어야 hard negative 구성 가능
        if len(labels_dict['entailment'])>0 and len(labels_dict['contradiction'])>0:
            pos = random.choice(list(labels_dict['entailment']))
            neg = random.choice(list(labels_dict['contradiction']))
            samples.append(InputExample(texts=[sent1, pos, neg]))

            # sent1을 가운데 두고, pos가 앞뒤로 바뀐 또다른 예시도 추가
            # (Optional - 다양하게 hard negative샘플을 늘릴 수 있음)
            samples.append(InputExample(texts=[pos, sent1, neg]))

    return samples

In [16]:
logging.info("Data Utilize(Pre-Processing) / Sample Extraction 완료")

In [17]:
def evaluate_on_sts_test(model_path, batch_size=64):
    """
    주어진 model_path에 있는 모델을 로드하고
    KorSTS Test 세트로 EmbeddingSimilarityEvaluator 시행 후
    최종 스코어를 반환합니다.
    """
    test_file_sts = os.path.join(KORSTS_PATH, "sts-test.tsv")
    test_samples_sts = load_kor_sts_samples(test_file_sts)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
        test_samples_sts,
        batch_size=batch_size,
        name='sts-test'
    )

    model = SentenceTransformer(model_path)
    score = test_evaluator(model)
    return score

In [18]:
import zipfile
import os

# 1. 압축 해제
zip_file_path = "/content/output.zip"
extract_path = "/content/output"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"압축 해제 완료: {extract_path}")

# 2. 원본 zip 파일 삭제
os.remove(zip_file_path)
print(f"{zip_file_path} 삭제 완료.")

# 3. 모델 경로 설정 (01-17 경로로 설정)
sts_model_path = "/content/output/sts-only-klue-roberta-base-01-17"

압축 해제 완료: /content/output
/content/output.zip 삭제 완료.


In [19]:
##################################################
# 실험 3: STS -> NLI                             #
##################################################
def train_sts_then_nli(
    sts_model_path,  # 실험 1에서 학습 완료된 모델 경로
    max_seq_length=128,
    batch_size=64,
    num_epochs=1,
    output_dir=OUTPUT_DIR
):
    """
    3) STS -> NLI 실험
       이미 STS로 학습된 모델(실험1 결과)을 불러와 NLI 학습 진행
    """
    model_save_path = os.path.join(
        output_dir,
        f"sts-to-nli-{datetime.now().strftime('%m-%d')}"
    )
    logging.info(f"[STS->NLI] 모델 세이브 경로: {model_save_path}")

    # 3-1) STS로 학습된 모델 불러오기
    model = SentenceTransformer(sts_model_path)
    logging.info(f"[STS->NLI] Pre-trained STS 모델 로드 완료: {sts_model_path}")

    # 3-2) KorNLI 데이터로 파인튜닝
    import glob
    train_files = glob.glob(os.path.join(KORNLI_PATH, "*train.ko.tsv"))
    train_samples_nli = []
    for train_file in train_files:
        logging.info(f"Loading NLI train file: {train_file}")
        train_samples_nli += load_kor_nli_samples(train_file)

    train_dataloader = datasets.NoDuplicatesDataLoader(train_samples_nli, batch_size=batch_size)
    train_loss = losses.MultipleNegativesRankingLoss(model)

    # dev 셋은 KorSTS dev
    dev_file = os.path.join(KORSTS_PATH, "sts-dev.tsv")
    dev_samples = load_kor_sts_samples(dev_file)
    dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=batch_size, name='sts-dev')

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
    logging.info(f"STS->NLI warmup steps: {warmup_steps}")

    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=dev_evaluator,
        epochs=num_epochs,
        evaluation_steps=1000,
        warmup_steps=warmup_steps,
        output_path=model_save_path
    )
    logging.info("[STS->NLI] 학습 완료!")
    return model_save_path

In [22]:
# -- 실험 3: STS -> NLI
sts_nli_path = train_sts_then_nli(
    sts_model_path=sts_model_path,
    max_seq_length=128,
    batch_size=64,
    num_epochs=1,
    output_dir=OUTPUT_DIR
)
final_scores['sts_then_nli'] = evaluate_on_sts_test(sts_nli_path, batch_size=64)
print(f"[STS->NLI] Test Score: {final_scores['sts_then_nli']}")
print(sts_nli_path)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
1000,0.645500,No log,0.852208,0.855738
2000,0.539900,No log,0.865140,0.866953
3000,0.497300,No log,0.852017,0.857557
4000,0.464500,No log,0.857561,0.860380
5000,0.430800,No log,0.856524,0.861034
6000,0.421100,No log,0.865185,0.867453
7000,0.394700,No log,0.862984,0.864563
8000,0.393600,No log,0.862447,0.863783
8885,0.381600,No log,0.862789,0.864369


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[STS->NLI] Test Score: {'sts-test_pearson_cosine': 0.8376511958206269, 'sts-test_spearman_cosine': 0.8489929165462522}
output/sts-to-nli-01-17


In [23]:
import shutil

# 압축할 폴더와 생성할 zip 파일 경로 설정
folder_to_zip = "/content/output"
zip_file_path = "/content/output.zip"

# 폴더를 압축
shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', folder_to_zip)
print(f"폴더가 압축되었습니다: {zip_file_path}")

폴더가 압축되었습니다: /content/output.zip
